## User Interface

The below interface allows a new user to create a new accout or to login if he is an existing user

And after the user is logged in then he can send messages which will be saved in the database

**SQLite is used for database**

In [1]:
import sqlite3
import bcrypt
import csv
import hashlib
import string
import pandas as pd



def encrypt_password(password):
    df = pd.read_csv('encryption_mapping.csv')
    encrypted_password = ''
    for char in password:
        # find the row in the conversion table that corresponds to the current character
        row = df.loc[df['USER TYPE'] == char.upper()]
        
        # if the character is not found in the table, skip it
        if len(row) == 0:
            encrypted_password += char
            continue
        
        # get the converted character from the table
        converted_char = row.iloc[0]['SYSTEM CONVERT']
        
        # append the converted character to the encrypted password
        encrypted_password += converted_char
    
    return encrypted_password

# function to create a new user
def create_user():
    email = input('Enter your email address: ')
    while True:
        password = input("Enter your password (must be alphanumeric): ")
        if all(char in string.ascii_letters + string.digits for char in password):
            break
    
    encrypted_password = encrypt_password(password)
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users (user_id INTEGER PRIMARY KEY AUTOINCREMENT, email_id TEXT UNIQUE, password TEXT)''')
    c.execute('''INSERT INTO users (email_id, password) VALUES (?, ?)''', (email, encrypted_password))
    conn.commit()
    c.close()
    print(f'Welcome {email}, please login to send a message.')

# function to authenticate an existing user
def authenticate_user():
    while True:
        email = input('Enter your email address: ')
        password = input('Enter your password: ')
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute('''SELECT user_id, email_id, password FROM users WHERE email_id = ?''', (email,))
        row = c.fetchone()
        if row is not None:
            user_id, email_id, encrypted_password = row
            if encrypt_password(password) == encrypted_password:
                print(f'Authentication successful. Welcome, {email}!')
                while True:
                    message = input('Enter your message (type "exit" to return to login page): ')
                    if message.lower() == 'exit':
                        break
                    c.execute('''CREATE TABLE IF NOT EXISTS messages (mssg_id INTEGER PRIMARY KEY AUTOINCREMENT, user_id INTEGER, mssg TEXT)''')
                    c.execute('''INSERT INTO messages (user_id, mssg) VALUES (?, ?)''', (user_id, message))
                    conn.commit()
                    print(f"Hi {email}, Message Sent")
                c.close()
                break
            else:
                print('Authentication failed. Please try again.')
        else:
            print('Authentication failed. Please try again.')
    return


# main function
def main():
    while True:
        user_type = input('Are you a new user (n) or an existing user (e).? Please type exit to quit  :')
        if user_type.lower() == 'n':
            create_user()
        elif user_type.lower() == 'e':
            authenticate_user()
        else:
            break

if __name__ == '__main__':
    main()


Are you a new user (n) or an existing user (e).? Please type exit to quit  :n
Enter your email address: shobhit.panwar@dcmail.ca
Enter your password (must be alphanumeric): ABC@123
Enter your password (must be alphanumeric): ABC123
Welcome shobhit.panwar@dcmail.ca, please login to send a message.
Are you a new user (n) or an existing user (e).? Please type exit to quit  :e
Enter your email address: shobhit.panwar@dcmail.ca
Enter your password: ABC123
Authentication successful. Welcome, shobhit.panwar@dcmail.ca!
Enter your message (type "exit" to return to login page): Hello! Welcome to my GitHub account!
Hi shobhit.panwar@dcmail.ca, Message Sent
Enter your message (type "exit" to return to login page): exit
Are you a new user (n) or an existing user (e).? Please type exit to quit  :exit


## The prediction model

The following code segment uses Multinomial Naive Bayes to train a model to predict if the message is SPAM or NOT SPAM

**It is a realtime code so it runs through all the messages as the database gets updated with new messages when the segment is executed**

In [2]:
import pandas as pd
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# load the labeled email data from the Excel file
df = pd.read_excel('file.xls')

# replace any NaN values in the email text with an empty string
df['email'].fillna('', inplace=True)

# separate the features (email text) from the labels (spam or not)
X = df['email']
y = df['label']

# vectorize the email text using a bag-of-words approach
vectorizer = CountVectorizer(stop_words='english')
X_vec = vectorizer.fit_transform(X)

# train a Naive Bayes model on the labeled data
model = MultinomialNB()
model.fit(X_vec, y)

# connect to the SQLite database
conn = sqlite3.connect('users.db')

# retrieve the messages and "from" email addresses from the database
c = conn.cursor()
c.execute('''SELECT u.email_id, mssg FROM messages m JOIN users u ON m.user_id = u.user_id''')
messages = [(row[0], row[1]) for row in c.fetchall()]
c.close()

# replace any NaN values in the messages with an empty string
messages = [(email, mssg if type(mssg) == str else '') for (email, mssg) in messages]

# vectorize the messages using the same vectorizer used for training
messages_vec = vectorizer.transform([mssg for (email, mssg) in messages])

# use the trained model to predict whether the messages are spam or not
predictions = model.predict(messages_vec)

# print the predictions for each message
for i, (email, mssg) in enumerate(messages):
    print(f'Message {i+1}: FROM: {email} - {"SPAM" if predictions[i] == 1 else "NOT SPAM"}')


Message 1: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 2: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 3: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 4: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 5: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 6: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 7: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 8: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 9: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 10: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 11: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 12: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 13: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 14: FROM: insaaf.saheed@gmail.com - NOT SPAM
Message 15: FROM: insaaf.saheed@gmail.com - SPAM
Message 16: FROM: insaaf.saheed@gmail.com - SPAM
Message 17: FROM: insaaf.saheed@gmail.com - SPAM
Message 18: FROM: insaaf.saheed@gmail.com - SPAM
Message 19: FROM: durham@gmail.com - NOT SPAM
Message 20: FROM: college